In [ ]:
#Vehicle Counting program

In [ ]:
import cv2
import numpy as np

#step1: read video file and display it
#step2: remove background
#step3: use erode,dilate,close operation remove noice
#step4: find contours
#step5: filter real car ,and draw trangle for car
#step6: draw a line,real car pass line counter car number
#step7: show result
#tips: when each step down,can,print rusult to check

cap = cv2.VideoCapture("car.mp4")

min_w = 90
min_h = 90
line_high = 550
cars = []
offset = 6
carno = 0
bgremove = cv2.createBackgroundSubtractorMOG2()
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))

def center(x,y,w,h):
  x1 = int(w/2)
  y1 = int(h/2)
  cx = x+x1
  cy = y+y1
  return cx,cy

while True:
  ret,frame = cap.read()

  if(ret == True):
    cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)#转灰度图
    blur = cv2.GaussianBlur(frame,(3,3),5)#去噪
    mask = bgremove.apply(blur)#去除背景
    erode = cv2.erode(mask,kernel,iterations=1)#腐蚀操作去除外部噪点
    dilate = cv2.dilate(erode,kernel,iterations=3)#膨胀操作还原放大
    close = cv2.morphologyEx(dilate,cv2.MORPH_CLOSE,kernel,iterations=2)#闭运算，去除所有内部噪点

    cnts,h = cv2.findContours(close,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#找到轮廓
    #画线
    cv2.line(frame,(10,line_high),(1280,line_high),(255,0,0),3)

    for (i,c) in enumerate(cnts):#使用轮廓画出矩形
      x,y,w,h = cv2.boundingRect(c)

      #对车宽高判断是不是车
      isValide = (w >= min_w) and (h >= min_h)
      if not isValide:
        continue
      #绘制有效的车
      cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
      cpoint = center(x,y,w,h)
      cars.append(cpoint)

      for (x,y) in cars:#画出线，车过线carno + 1
        if(y<(line_high+offset)) and (y>(line_high-offset)):
          carno += 1
          cv2.line(frame,(10,line_high),(1280,line_high),(0,0,255),3)
          cars.remove((x,y))
          print(carno)

    cv2.putText(frame,"vehicle count:"+str(carno),(450,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),5)
    cv2.imshow("frame",frame)



  key = cv2.waitKey(1)
  if(key == 27):
    break

cap.release()
cv2.destroyAllWindows()